In [1]:
# !pip install -r https://raw.githubusercontent.com/teddylee777/langchain-kr/main/requirements.txt

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Miniproject-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
Miniproject-RAG


In [4]:
import os, urllib.parse, requests
from dotenv import load_dotenv

load_dotenv()
cid = os.getenv("NAVER_CLIENT_ID")
csecret = os.getenv("NAVER_CLIENT_SECRET")

url = "https://openapi.naver.com/v1/search/news.json"
params = {"query": urllib.parse.quote("게임"), "display": 5, "sort": "date"}
headers = {"X-Naver-Client-Id": cid, "X-Naver-Client-Secret": csecret}

r = requests.get(url, headers=headers, params=params, timeout=10)
print(r.status_code, r.json().get("items", [])[:1])


200 [{'title': '콜라겐, 이 시간에 먹어야 효과 있다고? 영양제 흡수율, 효과 높이는 법', 'originallink': 'https://www.allurekorea.com/2025/07/25/%ec%bd%9c%eb%9d%bc%ea%b2%90-%ec%9d%b4-%ec%8b%9c%ea%b0%84%ec%97%90-%eb%a8%b9%ec%96%b4%ec%95%bc-%ed%9a%a8%ea%b3%bc-%ec%9e%88%eb%8b%a4%ea%b3%a0-%ec%98%81%ec%96%91%ec%a0%9c-%ed%9d%a1%ec%88%98/?utm_source=naver&utm_medium=partnership', 'link': 'https://www.allurekorea.com/2025/07/25/%ec%bd%9c%eb%9d%bc%ea%b2%90-%ec%9d%b4-%ec%8b%9c%ea%b0%84%ec%97%90-%eb%a8%b9%ec%96%b4%ec%95%bc-%ed%9a%a8%ea%b3%bc-%ec%9e%88%eb%8b%a4%ea%b3%a0-%ec%98%81%ec%96%91%ec%a0%9c-%ed%9d%a1%ec%88%98/?utm_source=naver&utm_medium=partnership', 'description': 'https://www.allurekorea.com/2025/03/08/%<b>EC%</b>9A%94%<b>EC%</b>A6%98-<b>%EA%B</b>1%B4<b>%EA%B</b>8%B0%<b>EC%</b>8B%9D-%<b>EC%</b>8B%9C%<b>EC%9E%</b>A5%<b>EC%</b>97%90%<b>EC%</b><b>84</b>%9C-%EB%A9%9C... ed%<b>84</b>%b0-%eb%8b%a4%<b>ec%</b>9d%b4%<b>ec%</b>86%<b>8c</b>-%<b>ec%</b>98%81%<b>ec%</b>96%91%<b>ec%</b>a0%9c-%<b>ec%</b>b6%94%<b>ec%</b><b>b2%</b>9c

In [ ]:
import os
import requests
from dotenv import load_dotenv
import re, html

load_dotenv()
CID = os.getenv("NAVER_CLIENT_ID")
CSECRET = os.getenv("NAVER_CLIENT_SECRET")

URL = "https://openapi.naver.com/v1/search/news.json"
HEADERS = {"X-Naver-Client-Id": CID, "X-Naver-Client-Secret": CSECRET}

def clean_text(s: str) -> str:
    if not s:
        return ""
    s = html.unescape(s)
    return re.sub(r"<.*?>", "", s).strip()  # <b>태그 제거

def filter_game_articles(items, keywords=None):
    if keywords is None:
        keywords = ["게임", "비디오게임", "e스포츠", "콘솔",
                    "닌텐도", "플레이스테이션", "Xbox", "스팀"]
    out = []
    for it in items:
        title = clean_text(it.get("title", ""))
        desc  = clean_text(it.get("description", ""))
        blob = f"{title} {desc}".lower()
        if any(k.lower() in blob for k in keywords):
            it["title"] = title
            it["description"] = desc
            out.append(it)
    return out

# ✅ 1) 먼저 필터 없이 원본이 나오는지 확인
params = {
    "query": "게임",     # ❗ urllib.parse.quote() 쓰지 말기
    "display": 50,       # 1~100
    "start": 1,          # 1~1000
    "sort": "date"       # 최신순
}
res = requests.get(URL, headers=HEADERS, params=params, timeout=10)

print("HTTP status:", res.status_code)
if res.status_code != 200:
    print(res.text)      # 오류 메시지 확인하고 종료
    raise SystemExit()

data = res.json()
items = data.get("items", [])
print("원본 개수:", len(items))

# ✅ 2) 이제 필터 적용
games = filter_game_articles(items)
print("게임 필터 후 개수:", len(games))

for a in games[:10]:
    title = a["title"]
    pub   = a.get("pubDate")
    link  = a.get("originallink") or a.get("link")
    print(f"- {title} | {pub}\n  {link}")


HTTP status: 200
원본 개수: 47
게임 필터 후 개수: 47
- 오픈게임재단 크로쓰(CORSS), 글로벌 거래소 엘뱅크(LBank) 상장 | Wed, 03 Sep 2025 10:34:00 +0900
  https://game.donga.com/119063/
- 펄어비스 '붉은사막' PAX 웨스트 성료, 이제 일본으로 간다 | Wed, 03 Sep 2025 10:34:00 +0900
  https://www.gamevu.co.kr/news/articleView.html?idxno=51398
- KT, 도심형 문화 행사 'K 페스타' 5일 개막 | Wed, 03 Sep 2025 10:34:00 +0900
  http://www.wsobi.com/news/articleView.html?idxno=288417
- [사이언스온고지신]거인의 어깨 위에서 피어난 창조적 파괴, '케데헌'이... | Wed, 03 Sep 2025 10:33:00 +0900
  https://www.etnews.com/20250903000146
- 애니팡4, 이용자 맞춤형 이벤트 ‘던져! 루비왕’ 첫 선 | Wed, 03 Sep 2025 10:32:00 +0900
  https://www.gameshot.net/common/con_view.php?code=GA68b799d6608c8
- 삼성전자, 하반기 매달 ‘신제품’..스마트폰 시장 게임 룰 바꿔 | Wed, 03 Sep 2025 10:32:00 +0900
  http://kpenews.com/View.aspx?No=3768211
- 넷마블 '뱀피르' 구글 매출 톱 10 실종 ... 말이되나 | Wed, 03 Sep 2025 10:32:00 +0900
  https://www.tgdaily.co.kr/news/articleView.html?idxno=342017
- 게임인재단, 디지털 법률교육 통해 미래 인재의 온라인 시민성 키운다 | Wed, 03 Sep 2025 10:32:00 +0900
  https://www.games

In [12]:
import os
import requests
import json
from dotenv import load_dotenv

# .env 파일에서 API 키 불러오기 (권장)
# .env 파일 안에는 다음처럼 저장해두세요:
# NAVER_CLIENT_ID=발급받은ID
# NAVER_CLIENT_SECRET=발급받은SECRET
load_dotenv()

CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

def get_naver_news(query, display=10, start=1, sort="sim"):
    """
    네이버 뉴스 검색 API 호출 함수
    query : 검색어
    display : 한 번에 가져올 뉴스 개수 (기본 10, 최대 100)
    start : 시작 위치 (기본 1, 최대 1000)
    sort : 정렬 옵션 (sim: 관련도순, date: 최신순)
    """
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,
        "start": start,
        "sort": sort
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error Code:", response.status_code)
        return None

# 사용 예시
if __name__ == "__main__":
    query = "고양이"
    result = get_naver_news(query, display=50)
    
    if result:
        for idx, item in enumerate(result['items'], 1):
            print(f"{idx}. {item['title']} ({item['link']})")


1. &quot;제2 케데헌 '더피', 카톡에 모였다&quot;…카카오, '힙트래디션 이모티콘' 출... (https://n.news.naver.com/mnews/article/003/0013455491?sid=105)
2. 집 비운 사이 발생한 불···범인은 바로 키우던 <b>고양이</b> (https://n.news.naver.com/mnews/article/032/0003393892?sid=102)
3. NCT 위시 유우시, <b>고양이</b> 변신완료 [포토엔HD] (https://m.entertain.naver.com/article/609/0001030248)
4. ‘<b>고양이</b>용 지하철’에 ‘야옹 맥도날드’까지…“3개월 완성” 유튜버... (https://n.news.naver.com/mnews/article/081/0003571052?sid=104)
5. 나린, <b>고양이</b> 볼 하트 (https://m.entertain.naver.com/article/421/0008462272)
6. 쿠팡 '9월 펫페어' 개최, 강아지·<b>고양이</b>·관상어·소동물 상품 4천여 종... (https://www.businesspost.co.kr/BP?command=article_view&num=410094)
7. [<b>고양이</b> 눈]“너를 보호해 줄게!” (https://n.news.naver.com/mnews/article/020/0003658296?sid=110)
8. [사진]NCT WISH, <b>고양이</b> 변신 (https://m.entertain.naver.com/article/109/0005383705)
9. [포토]미야오 나린, <b>고양이</b> 볼하트? (https://m.entertain.naver.com/article/241/0003463816)
10. 미야오 나린, '시크한 <b>고양이</b>처럼' (구찌 포토월) [HD포토] (https://www.topstarnews.net/news/articleView.html?